#IMPORT LIBRARIES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import random
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error as sklearn_mean_absolute_error
from sklearn.metrics import mean_squared_error as sklearn_mean_squared_error
from sklearn.metrics import r2_score as sklearn_r2_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

#IMPORT DATASET

In [ ]:
df = pd.read_csv('Desharnais.csv')
df.head(5)

,Project,TeamExp,ManagerExp,YearEnd,Length,Effort,Transactions,Entities,PointsNonAdjust,Adjustment,PointsAjust,Language
0,1,1,4,85,12,5152,253,52,305,34,302,1
1,2,0,0,86,4,5635,197,124,321,33,315,1
2,3,4,4,85,1,805,40,60,100,18,83,1
3,4,0,0,86,5,3829,200,119,319,30,303,1
4,5,0,0,86,4,2149,140,94,234,24,208,1


#TRAIN TEST DATASET

In [ ]:
from scipy.sparse import random
X=df.drop(columns=["Effort"])
y=df[['Effort']]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


# FEATURE SCALING

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Project          81 non-null     int64
 1   TeamExp          81 non-null     int64
 2   ManagerExp       81 non-null     int64
 3   YearEnd          81 non-null     int64
 4   Length           81 non-null     int64
 5   Effort           81 non-null     int64
 6   Transactions     81 non-null     int64
 7   Entities         81 non-null     int64
 8   PointsNonAdjust  81 non-null     int64
 9   Adjustment       81 non-null     int64
 10  PointsAjust      81 non-null     int64
 11  Language         81 non-null     int64
dtypes: int64(12)
memory usage: 7.7 KB


In [ ]:
df.describe()

,Project,TeamExp,ManagerExp,YearEnd,Length,Effort,Transactions,Entities,PointsNonAdjust,Adjustment,PointsAjust,Language
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,41.000000,2.185185,2.530864,85.740741,11.666667,5046.308642,182.123457,122.333333,304.456790,27.629630,289.234568,1.555556
std,23.526581,1.415195,1.643825,1.222475,7.424621,4418.767228,144.035098,84.882124,180.210159,10.591795,185.761088,0.707107
min,1.000000,-1.000000,-1.000000,82.000000,1.000000,546.000000,9.000000,7.000000,73.000000,5.000000,62.000000,1.000000
25%,21.000000,1.000000,1.000000,85.000000,6.000000,2352.000000,88.000000,57.000000,176.000000,20.000000,152.000000,1.000000
50%,41.000000,2.000000,3.000000,86.000000,10.000000,3647.000000,140.000000,99.000000,266.000000,28.000000,255.000000,1.000000
75%,61.000000,4.000000,4.000000,87.000000,14.000000,5922.000000,224.000000,169.000000,384.000000,35.000000,351.000000,2.000000
max,81.000000,4.000000,7.000000,88.000000,39.000000,23940.000000,886.000000,387.000000,1127.000000,52.000000,1116.000000,3.000000


#Stacked PCR

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor

# Apply PCA for dimensionality reduction
n_components = 5  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Define base regressors
base_regressors = [
    ('linear_regression', LinearRegression()),
    ('svr', SVR(kernel='linear')),
    ('random_forest', RandomForestRegressor(n_estimators=50, random_state=0))
]

# Define the meta-regressor
meta_regressor = LinearRegression()

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_pca, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_pca)

def mean_absolute_error(y_true, y_pred):
    mae = sklearn_mean_absolute_error(y_true, y_pred)
    maximum_value = 100000
    minimum_value = 0
    converted_value = mae / (maximum_value - minimum_value)
    return converted_value

def mean_squared_error(y_true, y_pred):
    mse = sklearn_mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    maximum_value = 100000
    minimum_value = 0
    converted_value = rmse / (maximum_value - minimum_value)
    return converted_value

def r2_score(y_true, y_pred):
    r2 = sklearn_r2_score(y_true, y_pred)
    maximum_value = 1
    minimum_value = 0
    converted_value = r2 / (maximum_value - minimum_value)
    return converted_value


# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.013007584896091037
Root Mean Squared Error (RMSE): 0.018928051870954973
R-squared (R2) Score: 0.018682587501879788


#Linear Regression
and hyper parameter tunuing

In [ ]:

# Hyperparameter Tuning
param_grid = {
    'fit_intercept': [True, False]

}

grid_search = GridSearchCV(LinearRegression(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Best parameters found
print("Best Parameters:", grid_search.best_params_)

# Evaluating the model with best parameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Best Parameters: {'fit_intercept': True}
Mean Absolute Error (MAE): 0.017365797324252173
Root Mean Squared Error (RMSE): 0.02361734736159871
R-squared (R2) Score: -0.5277767781322349
Mean Absolute Percentage Error (MMRE): 0.751294950580701


#Support Vector Machine

In [ ]:

#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)
from sklearn.svm import SVR
support=SVR(kernel='rbf')
support.fit(X_train,y_train)
y_pred=support.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.015511355498249193
Root Mean Squared Error (RMSE): 0.019520770353516726
R-squared (R2) Score: -0.04373819102520682
Mean Absolute Percentage Error (MMRE): 0.8967792499678072


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#Random Forest Regression

In [ ]:

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)
for i in np.arange(100,1100,100):
    random = RandomForestRegressor(n_estimators=i, max_features=5)
    random.fit(X_train, y_train)

    # Predict on the test set
    y_pred = random.predict(X_test)





# Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

<ipython-input-8-0ec458bb1753>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-8-0ec458bb1753>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-8-0ec458bb1753>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-8-0ec458bb1753>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-8-0ec458bb1753>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

Mean Absolute Error (MAE): 0.010844527647058825
Root Mean Squared Error (RMSE): 0.014533111039883051
R-squared (R2) Score: 0.4214846940172209
Mean Absolute Percentage Error (MMRE): 0.5443024844383682


#KNeighbor


In [ ]:
# Splitting the dataset into features (X) and target variable (y)
X = df.drop(columns=['Effort'])
y = df['Effort']

# Splitting the dataset into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)

# Instantiate and fit the KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3, weights='uniform')
neigh.fit(X_train, y_train)

# Predict on the test data
y_pred = neigh.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.016528039215686276
Root Mean Squared Error (RMSE): 0.02607993317436003
R-squared (R2) Score: -0.8629904072146677
Mean Absolute Percentage Error (MMRE): 0.7768726389842765


#Bagging

In [ ]:

from sklearn.ensemble import BaggingRegressor

# Replace DecisionTreeRegressor with BaggingRegressor
bagging_dtree = BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=22),
                                 n_estimators=10,  # You can adjust the number of base estimators (trees)
                                 random_state=22)
bagging_dtree.fit(X_train, y_train)
y_pred = bagging_dtree.predict(X_test)


# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.013465882352941176
Root Mean Squared Error (RMSE): 0.018599026543153842
R-squared (R2) Score: 0.05250244359882261
Mean Absolute Percentage Error (MMRE): 0.5811182426309646


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


#Boosting

In [ ]:

gradient_booster = GradientBoostingRegressor(learning_rate=0.1)
gradient_booster.fit(X_train,y_train)
y_pred = gradient_booster.predict(X_test)


# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.00946418319216312
Root Mean Squared Error (RMSE): 0.012517008032323492
R-squared (R2) Score: 0.5708602617161413
Mean Absolute Percentage Error (MMRE): 0.4737425112014018


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#CATBOOST

In [ ]:
!pip install catboost
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Instantiate the CatBoostRegressor model
catboost_model = CatBoostRegressor(iterations=100, depth=8, learning_rate=0.1, loss_function='MAE', random_seed=0)

# Fit the model to the training data
catboost_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = catboost_model.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


0:	learn: 2395.2499969	total: 47.3ms	remaining: 4.68s
1:	learn: 2221.8709942	total: 48.9ms	remaining: 2.4s
2:	learn: 2061.9877933	total: 50.5ms	remaining: 1.63s
3:	learn: 1901.1201770	total: 52ms	remaining: 1.25s
4:	learn: 1775.5877475	total: 53.5ms	remaining: 1.02s
5:	learn: 1696.0786843	total: 55.1ms	remaining: 863ms
6:	learn: 1606.4738447	total: 56.5ms	remaining: 751ms
7:	learn: 1498.2582896	total: 58ms	remaining: 667ms
8:	learn: 1434.6388812	total: 59.4ms	remaining: 601ms
9:	learn: 1342.0438170	total: 60.8ms	remaining: 548ms
10:	learn: 1295.7886676	total: 62.3ms	remaining: 504ms
11:	learn: 1209.3602444	total: 63.7ms	remaining: 467ms
12:	learn: 1157.3515678	total: 65.3ms	remaining: 437ms
13:	learn: 1137.3411035	total: 66ms	remaining: 405ms
14:	learn: 1085.1268953	total: 67.7ms	remaining: 384ms
15:	learn: 1057.3090460	total: 68.3ms	remaining: 358ms
16:	learn: 1005.2876186	total: 70.5ms	remaining: 344ms
17:	learn: 982.5128759	total: 71.2ms	remaining: 325ms
18:	learn: 932.6221229	total

# Stacked Rpart


In [ ]:

# Define base regressors
base_regressors = [
    ('decision_tree', DecisionTreeRegressor(max_depth=5, random_state=0)),
    ('linear_regression', LinearRegression()),
    ('svr', SVR(kernel='linear')),
    ('random_forest', RandomForestRegressor(n_estimators=50, random_state=0))
]

# Define the meta-regressor
meta_regressor = DecisionTreeRegressor()

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Error (MAE): 0.4641190476190476
Root Mean Squared Error (RMSE): 0.7300215049017719
R-squared (R2) Score: -0.4211754149317497


#Stacking using decision tree


In [ ]:

from sklearn.tree import DecisionTreeRegressor


# Define base models
dt1 = DecisionTreeRegressor(max_depth=3, random_state=42)
dt2 = DecisionTreeRegressor(max_depth=5, random_state=42)

# Fit the base models on the training data
dt1.fit(X_train, y_train)
dt2.fit(X_train, y_train)

# Make predictions on the test data using the base models
dt1_y_pred = dt1.predict(X_test)
dt2_y_pred = dt2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((dt1.predict(X_train), dt2.predict(X_train)))
stacking_test = np.column_stack((dt1_y_pred, dt2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

<ipython-input-12-a50da682a858>:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


Mean Absolute Error (MAE): 0.014277655218180075
Root Mean Squared Error (RMSE): 0.0284769362985607
R-squared (R2) Score: -1.2211822673678911
Mean Absolute Percentage Error (MMRE): 0.5146323880970473


# **Stacking using SVM**

In [ ]:




# Define base models
svm1 = SVR(kernel='linear')
svm2 = SVR(kernel='rbf')

# Fit the base models on the training data
svm1.fit(X_train, y_train)
svm2.fit(X_train, y_train)

# Make predictions on the test data using the base models
svm1_y_pred = svm1.predict(X_test)
svm2_y_pred = svm2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((svm1.predict(X_train), svm2.predict(X_train)))
stacking_test = np.column_stack((svm1_y_pred, svm2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-13-1a0177b7bc32>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


Mean Absolute Error (MAE): 0.01421525882352941
Root Mean Squared Error (RMSE): 0.018791589532961124
R-squared (R2) Score: 0.032781253860381665
Mean Absolute Percentage Error (MMRE): 0.5947344939524469


# **Stacking using Random Forest**

In [ ]:




# Define base models
rf1 = RandomForestRegressor(n_estimators=10, random_state=42)
rf2 = RandomForestRegressor(n_estimators=50, random_state=42)

# Fit the base models on the training data
rf1.fit(X_train, y_train)
rf2.fit(X_train, y_train)

# Make predictions on the test data using the base models
rf1_y_pred = rf1.predict(X_test)
rf2_y_pred = rf2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((rf1.predict(X_train), rf2.predict(X_train)))
stacking_test = np.column_stack((rf1_y_pred, rf2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

<ipython-input-14-357b028b61d0>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf1.fit(X_train, y_train)
<ipython-input-14-357b028b61d0>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf2.fit(X_train, y_train)
<ipython-input-14-357b028b61d0>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


Mean Absolute Error (MAE): 0.013273817647058825
Root Mean Squared Error (RMSE): 0.017492469939259073
R-squared (R2) Score: 0.16189208392461452
Mean Absolute Percentage Error (MMRE): 0.5152526203119923


# Stacked XGBoost

In [ ]:
from xgboost import XGBRegressor


# Create the base models (XGBoost)
xgb_model1 = XGBRegressor(n_estimators=100, random_state=42)
xgb_model2 = XGBRegressor(n_estimators=100, random_state=42)

# Train the base models
xgb_model1.fit(X_train, y_train)
xgb_model2.fit(X_train, y_train)

# Make predictions with the base models
xgb_predictions1 = xgb_model1.predict(X_test)
xgb_predictions2 = xgb_model2.predict(X_test)

# Create a meta-model (XGBoost) to combine predictions
meta_model = XGBRegressor(n_estimators=100, random_state=42)

# Combine the predictions from base models as meta-features
meta_features = pd.DataFrame({'XGB1': xgb_predictions1, 'XGB2': xgb_predictions2})

# Train the meta-model on the predictions
meta_model.fit(meta_features, y_test)

# Make predictions with the meta-model
meta_predictions = meta_model.predict(meta_features)

# Calculate the MAE for the ensemble model
mae = mean_absolute_error(y_test, meta_predictions)
mse = mean_squared_error(y_test, meta_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, meta_predictions)

print(f"Ensemble MAE: {mae}")

print(f"Ensemble RMSE: {rmse}")
print(f"Ensemble R-squared: {r2}")
mmre = mean_absolute_percentage_error(y_test, meta_predictions)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Ensemble MAE: 9.334788602941177e-09
Ensemble RMSE: 0.00011188830028511695
Ensemble R-squared: 0.9999999999995707
Mean Absolute Percentage Error (MMRE): 4.745079074911728e-07


# Stacked GradientBoost

In [ ]:
# Define base models
base_model1 = GradientBoostingRegressor(n_estimators=100, random_state=42)
base_model2 = GradientBoostingRegressor(n_estimators=200, random_state=42)

# Define the stacked model
stacked_model = StackingRegressor(
    estimators=[
        ('model1', base_model1),
        ('model2', base_model2)
    ],
    final_estimator=GradientBoostingRegressor(n_estimators=50, random_state=42)
)

# Fit the stacked model
stacked_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = stacked_model.predict(X_test)


# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.011109166652156978
Root Mean Squared Error (RMSE): 0.015150446887628139
R-squared (R2) Score: 0.3712926223160864
Mean Absolute Percentage Error (MMRE): 0.5625301564284609


#Stacked nnet

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor

# Define base regressors (neural networks)
base_regressors = [
    ('mlp_1', MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)),
    ('mlp_2', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=0)),
    ('mlp_3', MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=1000, random_state=0))
]

# Define the meta-regressor
meta_regressor = MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklear

Mean Absolute Error (MAE): 0.014874011463311226
Root Mean Squared Error (RMSE): 0.01822362760468721
R-squared (R2) Score: 0.0903646395915807
Mean Absolute Percentage Error (MMRE): 0.5119678769403706


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


#Stacked glmnet

In [ ]:
from sklearn.linear_model import ElasticNet


# Define base regressors (GLMNet - ElasticNet)
base_regressors = [
    ('elastic_net_1', ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)),
    ('elastic_net_2', ElasticNet(alpha=0.01, l1_ratio=0.7, random_state=0)),
    ('elastic_net_3', ElasticNet(alpha=0.001, l1_ratio=0.9, random_state=0))
]

# Define the meta-regressor
meta_regressor = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) # RMSE

r2 = r2_score(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)
mmre = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MMRE):", mmre)

Mean Absolute Error (MAE): 0.012421422436833945
Root Mean Squared Error (RMSE): 0.017992605414965425
R-squared (R2) Score: 0.11328147480551076
Mean Absolute Percentage Error (MMRE): 0.3880388238445624


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.440e+08, tolerance: 1.445e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.808e+08, tolerance: 1.445e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-package

#Averaging

In [ ]:


# Define base models
models = [
    RandomForestRegressor(random_state=42),
    LinearRegression()
]

# Train base models and make predictions
predictions = []
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions.append(y_pred)

# Average predictions
avg_predictions = sum(predictions) / len(predictions)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, avg_predictions)
rmse = r_mse(y_test, avg_predictions) # RMSE
r2 = r_squared(y_test, avg_predictions)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

Mean Absolute Error (MAE): 0.2122038783415238
Root Mean Squared Error (RMSE): 0.3274440836896902
R-squared (R2) Score: 0.7140759581589045
